### Activating KVM on Jetson Nano

This is based on https://developer.ridgerun.com/wiki/index.php?title=Jetson_Nano/Development/Building_the_Kernel_from_Source

Jetson Nano's original image does not come with KVM enabled, thus we have to recompile the kernel and activate it. In this article, we're gonna do everything inside the Nano so we don't have to take out the SD card or flash a new image. You might be surprised that recompiling the kernel in the Nano itself takes less than 30 minutes.

So, put your Nano to work with the latest Ubuntu image provided by Jetson Nano, and then boot it and install the dependencies needed to build the kernel:

In [ ]:
# Installs dependencies for getting/building the kernel
sudo apt update && sudo apt-get install -y build-essential bc git curl wget xxd kmod libssl-dev

Now, we should get the kernel source at [developer.nvidia.com](https://developer.nvidia.com/embedded/downloads). As af today, (july 15) the latest is [r32_release_v5.1/sources/t210](https://developer.nvidia.com/embedded/l4t/r32_release_v5.1/r32_release_v5.1/sources/t210/public_sources.tbz2). But wait, use the script below to download and unpack everything.

#### Documents

- [Getting Started Page](https://developer.nvidia.com/embedded/learn/get-started-jetson-nano-devkit#write): [Jetson Nano Developer Kit SD Card Image 📦](https://developer.nvidia.com/jetson-nano-sd-card-image)
  - [r32_Release_v7.1/JP_4.6.1_b110_SD_Card/Jeston_Nano/jetson-nano-jp461-sd-card-image.zip 📦 (redirect)](https://developer.download.nvidia.com/embedded/L4T/r32_Release_v7.1/JP_4.6.1_b110_SD_Card/Jeston_Nano/jetson-nano-jp461-sd-card-image.zip)
- [JetPack Archive
](https://developer.nvidia.com/embedded/jetpack-archive)
  - **[JetPack 4.6.1](https://developer.nvidia.com/embedded/jetpack-sdk-461)**: [r32_release_v7.1/jp_4.6.1_b110_sd_card/jeston_nano/jetson-nano-jp461-sd-card-image.zip 📦](https://developer.nvidia.com/embedded/l4t/r32_release_v7.1/jp_4.6.1_b110_sd_card/jeston_nano/jetson-nano-jp461-sd-card-image.zip)
    - **[Jetson Linux R32.7.1](https://developer.nvidia.com/embedded/linux-tegra-r3271)**
      - **L4T Driver Package (BSP) Sources**:
        - [r32_Release_v7.1/Sources/T210/public_sources.tbz2 📦](https://developer.download.nvidia.com/embedded/L4T/r32_Release_v7.1/Sources/T210/public_sources.tbz2)
  <!-- - **[JetPack SDK 4.6.3](https://developer.nvidia.com/jetpack-sdk-463)**: (JetPack 4.6.3 can be installed using SDK Manager.)
    - **[Jetson Linux R32.7.3](https://developer.nvidia.com/embedded/linux-tegra-r3273)**
      - **Driver Package (BSP) Sources**:
        - [r32_release_v73_sources_t210_public_sources_tbz2 📦](https://developer.nvidia.com/downloads/remack-sdksjetpack-463r32releasev73sourcest210publicsourcestbz2)
  - **[JetPack SDK 4.6.5](https://developer.nvidia.com/jetpack-sdk-465)**
    - **[Jetson Linux R32.7.5](https://developer.nvidia.com/embedded/linux-tegra-r3275)**
      - **Developer Guide**:
        - [NVIDIA Jetson Linux Developer Guide : Introduction (32.7.5)](https://docs.nvidia.com/jetson/archives/l4t-archived/l4t-3275/index.html) -->
  - **[JetPack SDK 4.6.6](https://developer.nvidia.com/jetpack-sdk-466)**: (JetPack 4.6.6 can be installed using SDK Manager.)
    - **[Jetson Linux R32.7.6](https://developer.nvidia.com/embedded/linux-tegra-r3276)** - *final release for Jetson Linux R32 and JetPack 4*
      - **Driver Package (BSP) Sources**:
          <!-- - [r32_release_v7.6/sources/t210/public_sources.tbz2 📦](https://developer.nvidia.com/downloads/embedded/l4t/r32_release_v7.6/sources/t210/public_sources.tbz2) -->
          - [r32_Release_v7.6/sources/T210/public_sources.tbz2 📦](https://developer.download.nvidia.com/embedded/L4T/r32_Release_v7.6/sources/T210/public_sources.tbz2)
      - **Developer Guide**: [NVIDIA Jetson Linux Developer Guide : Introduction (32.7.6)](https://docs.nvidia.com/jetson/archives/l4t-archived/l4t-3276/index.html)
        - [NVIDIA Jetson Linux Developer Guide : Kernel Customization (32.7.6)](https://docs.nvidia.com/jetson/archives/l4t-archived/l4t-3276/index.html#page/Tegra%20Linux%20Driver%20Package%20Development%20Guide/kernel_custom.html#)
  - **JetPack SDK Common**:
    - **Jetson Linux Common**:
      - **GCC 7.3.1 for 64 bit BSP and Kernel**:
        - [gcc-linaro-7.3.1-2018.05-x86_64_aarch64-linux-gnu.tar.xz 📦](https://developer.nvidia.com/embedded/dlc/l4t-gcc-7-3-1-toolchain-64-bit)
      - **Sources for the GCC 7.3.1 Tool Chain for 64-bit BSP and Kernel**:
        - [gcc-linaro-7.3-2018.05.tar.xz 📦](https://developer.nvidia.com/gcc-linaro-731-201805-sources)

#### Environments

In [ ]:
# 現在動いているSDイメージとカーネルのバージョンを確認する
cat /etc/os-release
# =>
# NAME="Ubuntu"
# VERSION="18.04.6 LTS (Bionic Beaver)"
# ID=ubuntu
# ID_LIKE=debian
# PRETTY_NAME="Ubuntu 18.04.6 LTS"
# VERSION_ID="18.04"
# HOME_URL="https://www.ubuntu.com/"
# SUPPORT_URL="https://help.ubuntu.com/"
# BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
# PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
# VERSION_CODENAME=bionic
# UBUNTU_CODENAME=bionic

In [ ]:
cat /etc/nv_tegra_release
# => # R32 (release), REVISION: 7.3, GCID: 31982016, BOARD: t210ref, EABI: aarch64, DATE: Tue Nov 22 17:30:08 UTC 2022
# => # R32 (release), REVISION: 7.6, GCID: 38171779, BOARD: t210ref, EABI: aarch64, DATE: Tue Nov  5 07:46:14 UTC 2024

In [ ]:
cat /proc/version
# => Linux version 4.9.299-tegra (buildbrain@mobile-u64-5333-d8000) (gcc version 7.3.1 20180425 [linaro-7.3-2018.05 revision d29120a424ecfbc167ef90065c0eeb7f91977701] (Linaro GCC 7.3-2018.05) ) #1 SMP PREEMPT Tue Nov 22 09:24:39 PST 2022
# => Linux version 4.9.299-tegra (jetson@jetson-nano) (gcc version 7.5.0 (Ubuntu/Linaro 7.5.0-3ubuntu1~18.04) ) #1 SMP PREEMPT Sat Jun 17 16:52:07 JST 2023
# => Linux version 4.9.299-tegra (codespace@codespaces-600304) (gcc version 7.3.1 20180425 [linaro-7.3-2018.05 revision d29120a424ecfbc167ef90065c0eeb7f91977701] (Linaro GCC 7.3-2018.05) ) #1 SMP PREEMPT Sun Jun 18 04:26:14 UTC 2023
# => Linux version 4.9.337-tegra (buildbrain@mobile-u64-5499-d7000) (gcc version 7.3.1 20180425 [linaro-7.3-2018.05 revision d29120a424ecfbc167ef90065c0eeb7f91977701] (Linaro GCC 7.3-2018.05) ) #1 SMP PREEMPT Mon Nov 4 23:41:41 PST 2024

In [ ]:
uname -a
# => Linux jetson-nano 4.9.299-tegra #1 SMP PREEMPT Tue Nov 22 09:24:39 PST 2022 aarch64 aarch64 aarch64 GNU/Linux
# => Linux jetson-nano 4.9.299-tegra #1 SMP PREEMPT Sat Jun 17 16:52:07 JST 2023 aarch64 aarch64 aarch64 GNU/Linux
# => Linux jetson-nano 4.9.299-tegra #1 SMP PREEMPT Sun Jun 18 04:26:14 UTC 2023 aarch64 aarch64 aarch64 GNU/Linux
# => Linux jetson-nano 4.9.337-tegra #1 SMP PREEMPT Mon Nov 4 23:41:41 PST 2024 aarch64 aarch64 aarch64 GNU/Linux

#### Workspace settings

In [ ]:
#export BASEDIR=~
#export BASEDIR=/workspaces/jetson_nano_kvm
export BASEDIR=~/jetson_nano_kvm
export JETSON_NANO_KERNEL_SOURCE=${BASEDIR:?}/Linux_for_Tegra/source/public

In [ ]:
# 1.Set the shell variable with the command:
export TEGRA_KERNEL_OUT=${JETSON_NANO_KERNEL_SOURCE:?}/build
# Where:
#   <outdir> is the desired destination for the compiled kernel.
export KERNEL_MODULES_OUT=${JETSON_NANO_KERNEL_SOURCE:?}/modules

In [ ]:
# 2.If cross-compiling on a non-Jetson system, export the following environment variables:
# export CROSS_COMPILE="$BASEDIR"/l4t-gcc/gcc-linaro-7.3.1-2018.05-x86_64_aarch64-linux-gnu/bin/aarch64-linux-gnu-
# export LOCALVERSION=-tegra
# Where:
#   <cross_prefix> is the absolute path of the ARM64 toolchain without the gcc suffix.
#   For example, for the reference ARM64 toolchain, <cross_prefix> is:
#     <toolchain_install_path>/bin/aarch64-linux-gnu-
# See The L4T Toolchain for information on how to download and build the reference toolchains.
# Note: NVIDIA recommends using the Linaro 7.3.1 2018.05 toolchain.

#### Cleanup

In [ ]:
[ ! -d "${BASEDIR:?}" ] && mkdir -p "$BASEDIR"
cd "$BASEDIR"
ls -A "$BASEDIR"
# cp "${TEGRA_KERNEL_OUT:?}"/.config.template $BASEDIR/

In [ ]:
rm -rf "${JETSON_NANO_KERNEL_SOURCE:?}"
rm -rf "$BASEDIR/l4t-gcc"
rm "$BASEDIR"/public_sources.tbz2
rm "$BASEDIR"/kernel*.tar.gz

In [ ]:
env | grep -E 'BASEDIR|JETSON_NANO_KERNEL_SOURCE|TEGRA_KERNEL_OUT|KERNEL_MODULES_OUT|CROSS_COMPILE|LOCALVERSION' | sort

#### Linux kernel

The linux kernel has a config file which dictates which kernel options are enabled in the compilation process. What we need to do is enable these options, which are

```sh
CONFIG_KVM=y
CONFIG_VHOST_NET=m
```

When uncompressed, the `public_sources.tbz2` file will appear at `Linux_for_Tegra`. We also need to unpack at `Linux_for_Tegra/source/public/kernel_src.tbz2`.
The config file for tegra is at `Linux_for_Tegra/source/public/kernel/kernel-4.9/arch/arm64/configs/tegra_defconfig`

So let's do all of this in one shot. Remember that you'd have to change the kernel version and the link if you want newer kernels, and you should pick the kernel that matches your release for better compatibility. So:

#### クロスコンパイル時にのみ必要？

In [ ]:
# #wget -O $BASEDIR/gcc-linaro-7.3.1-2018.05-x86_64_aarch64-linux-gnu.tar.xz http://releases.linaro.org/components/toolchain/binaries/7.3-2018.05/aarch64-linux-gnu/gcc-linaro-7.3.1-2018.05-x86_64_aarch64-linux-gnu.tar.xz
# wget -O $BASEDIR/gcc-linaro-7.3.1-2018.05-x86_64_aarch64-linux-gnu.tar.xz https://developer.nvidia.com/embedded/dlc/l4t-gcc-7-3-1-toolchain-64-bit

# mkdir "$BASEDIR/l4t-gcc"
# cd "$BASEDIR/l4t-gcc"
# tar -xf $BASEDIR/gcc-linaro-7.3.1-2018.05-x86_64_aarch64-linux-gnu.tar.xz

In [ ]:
# Gets the kernel
#wget https://developer.download.nvidia.com/embedded/L4T/r32_Release_v7.1/Sources/T210/public_sources.tbz2  # 32.7.1
#wget -O "$BASEDIR"/public_sources.tbz2 https://developer.nvidia.com/downloads/remack-sdksjetpack-463r32releasev73sourcest210publicsourcestbz2 # 32.7.3
wget -O "$BASEDIR"/public_sources.tbz2 https://developer.download.nvidia.com/embedded/L4T/r32_Release_v7.6/sources/T210/public_sources.tbz2 # 32.7.6
cd "$BASEDIR"
tar -xf "$BASEDIR"/public_sources.tbz2

ls -AF $JETSON_NANO_KERNEL_SOURCE
cd $JETSON_NANO_KERNEL_SOURCE
ls -lAF $JETSON_NANO_KERNEL_SOURCE/kernel_src.tbz2
tar -xf $JETSON_NANO_KERNEL_SOURCE/kernel_src.tbz2
# =>
# hardware/
# kernel/
# nvbuild.sh
# nvcommon_build.sh

# Applies the new configs to tegra_defconfig so KVM option is enabled
ls -AF $JETSON_NANO_KERNEL_SOURCE/kernel/kernel-4.9
cd $JETSON_NANO_KERNEL_SOURCE/kernel/kernel-4.9
# echo "CONFIG_KVM=y
# CONFIG_VHOST_NET=m" >> arch/arm64/configs/tegra_defconfig

Compiling the kernel now would already activate KVM, but we would still miss an important feature that makes virtualization much faster: the irq chip. Without it, virtualization is still possible but an emulated irq chip is much slower. On `firecracker` (a virtualization tool written by AWS), it will not work as it requires this.

What we need to do is specify, in the device tree, the features of the irq chip on the CPU. The device tree is a file that contains addresses for all devices on the Jetson Nano chip.

This must be done by hand. Apply the patch below to the file `Linux_for_Tegra/source/public/kernel_src/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-bthrot-cdev.dtsi`. Don't use the patch tool as it'll likely not work, just do it by hand:

In [ ]:
grep -A5 '0x0 0x50041000 0x0 0x1000' -r $JETSON_NANO_KERNEL_SOURCE/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc
# => tegra210-soc-base.dtsi

grep: /hardware/nvidia/soc/t210/kernel-dts/tegra210-soc: No such file or directory


: 2

In [ ]:
# cp -a $JETSON_NANO_KERNEL_SOURCE/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi{,.orig}
sed -i.orig -e 's/0x0 0x50042000 0x0 0x0100>;/0x0 0x50042000 0x0 0x2000\n\t\t       0x0 0x50044000 0x0 0x2000\n\t\t       0x0 0x50046000 0x0 0x2000>;\n\t\tinterrupts = <GIC_PPI 9 (GIC_CPU_MASK_SIMPLE(4) | IRQ_TYPE_LEVEL_HIGH)>;/' \
  $JETSON_NANO_KERNEL_SOURCE/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi
# Restore:
#   mv $JETSON_NANO_KERNEL_SOURCE/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi{.orig,}

In [ ]:
diff -u $JETSON_NANO_KERNEL_SOURCE/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi{.orig,}
# --- a/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi.orig     2023-06-17 16:22:40.227646924 +0900
# +++ b/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi          2023-06-17 16:24:40.669522766 +0900
# @@ -351,7 +351,10 @@
#                 #interrupt-cells = <3>;
#                 interrupt-controller;
#                 reg = <0x0 0x50041000 0x0 0x1000
# -                      0x0 0x50042000 0x0 0x0100>;
# +                      0x0 0x50042000 0x0 0x2000
# +                      0x0 0x50044000 0x0 0x2000
# +                      0x0 0x50046000 0x0 0x2000>;
# +               interrupts = <GIC_PPI 9 (GIC_CPU_MASK_SIMPLE(4) | IRQ_TYPE_LEVEL_HIGH)>;
#                 status = "disabled";
#         };


```diff
--- a/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi     2020-08-31 08:40:36.602176618 +0800
+++ b/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi     2020-08-31 08:41:45.223679918 +0800
@@ -351,7 +351,10 @@
                #interrupt-cells = <3>;
                interrupt-controller;
                reg = <0x0 0x50041000 0x0 0x1000
-                      0x0 0x50042000 0x0 0x0100>;
+                       0x0 0x50042000 0x0 0x2000
+                       0x0 0x50044000 0x0 0x2000
+                       0x0 0x50046000 0x0 0x2000>;
+               interrupts = <GIC_PPI 9 (GIC_CPU_MASK_SIMPLE(4) | IRQ_TYPE_LEVEL_HIGH)>;
                status = "disabled";
        };
```

as you see, we added more `reg` and `interrupts`. Now, when we compile the kernel image, we'll also compile device tree files from this `dsti` file.

[jetson nanoのカーネルをソースからビルドする (zenn.dev)](https://zenn.dev/tetsu_koba/articles/7d49c86da7a4b0)

Now we should compile everything:

In [ ]:
cd $JETSON_NANO_KERNEL_SOURCE/..

In [ ]:
[ ! -d "${JETSON_NANO_KERNEL_SOURCE}.orig" ] && rsync -avh --delete "${JETSON_NANO_KERNEL_SOURCE}/" "${JETSON_NANO_KERNEL_SOURCE}.orig"

In [ ]:
ls -lA

In [ ]:
cd $JETSON_NANO_KERNEL_SOURCE
# Generates the config file (you should manually enable/disable some missing by pressing y/n and enter)
# ARCH が提供する defconfig ファイルのデフォルト値で新しい設定ファイルを生成します。ソースに付属していたデフォルト設定ファイルに戻すにはこのオプションを使用してください。
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" tegra_defconfig
ls -la "${TEGRA_KERNEL_OUT:?}"/.config*
cp -a "${TEGRA_KERNEL_OUT:?}"/.config{,.orig}
#cp "${BASEDIR:?}"/.config.template "${TEGRA_KERNEL_OUT:?}"/

In [ ]:
# https://zenn.dev/nkte8/articles/2021-12-06-r01#%E8%A8%AD%E5%AE%9A%E6%96%B9%E6%B3%95

# コンフィグをカスタマイズ (interactive)
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" menuconfig
# ncurses ベースの擬似グラフィカルメニュー(テキスト入力のみ)。メニュー内を移動して希望するオプションを変更します。
#make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" nconfig
# Symbol: KVM [=n]
#   │ Type  : boolean
#   │ Prompt: Kernel-based Virtual Machine (KVM) support
#   │   Location:
#   │ (1) -> Virtualization (VIRTUALIZATION [=y])
#   │   Defined at arch/arm64/kvm/Kconfig:22
#   │   Depends on: VIRTUALIZATION [=y] && OF [=y]
#   │   Selects: MMU_NOTIFIER [=y] && PREEMPT_NOTIFIERS [=n] && ANON_INODES [=y] && HAVE_KVM_CPU_RELAX_INTERCEPT [=n] && HAVE_KVM_ARCH_TLB_FLUSH_ALL [=n] && KVM_MMIO [=n] && KVM_ARM_HOST [ │

# Symbol: VHOST_NET [=n]
#   │ Type  : tristate
#   │ Prompt: Host kernel accelerator for virtio net
#   │   Location:
#   │ (1) -> Virtualization (VIRTUALIZATION [=y])
#   │   Defined at drivers/vhost/Kconfig:1
#   │   Depends on: VIRTUALIZATION [=y] && NET [=y] && EVENTFD [=y] && (TUN [=y] || !TUN [=y]) && (MACVTAP [=m] || !MACVTAP [=m])
#   │   Selects: VHOST [=n]

# Symbol: VHOST_VSOCK [=n]
#   │ Type  : tristate
#   │ Prompt: vhost virtio-vsock driver
#   │   Location:
#   │ (1) -> Virtualization (VIRTUALIZATION [=y])
#   │   Defined at drivers/vhost/Kconfig:22
#   │   Depends on: VIRTUALIZATION [=y] && VSOCKETS [=n] && EVENTFD [=y]
#   │   Selects: VIRTIO_VSOCKETS_COMMON [=n] && VHOST [=n]

# Symbol: VSOCKETS [=n]
#   │ Type  : tristate
#   │ Prompt: Virtual Socket protocol
#   │   Location:
#   │     -> Networking support (NET [=y])
#   │ (1)   -> Networking options
#   │   Defined at net/vmw_vsock/Kconfig:5
#   │   Depends on: NET [=y]

# Symbol: MACVTAP [=m]
#   │ Type  : tristate
#   │ Prompt: MAC-VLAN based tap driver
#   │   Location:
#   │     -> Device Drivers
#   │       -> Network device support (NETDEVICES [=y])
#   │         -> Network core driver support (NET_CORE [=y])
#   │ (1)       -> MAC-VLAN support (MACVLAN [=m])
#   │   Defined at drivers/net/Kconfig:134
#   │   Depends on: NETDEVICES [=y] && NET_CORE [=y] && MACVLAN [=m] && INET [=y]

In [ ]:
# RT_GROUP_SCHED
# | Symbol: RT_GROUP_SCHED [=n]
# | Type  : boolean
# | Prompt: Group scheduling for SCHED_RR/FIFO
# |   Location:
# |     -> General setup
# |       -> Control Group support (CGROUPS [=y])
# | (1)     -> CPU controller (CGROUP_SCHED [=y])
# |   Defined at init/Kconfig:1191
# |   Depends on: CGROUPS [=y] && CGROUP_SCHED [=y]

# IP_SET
# | Symbol: IP_SET [=n]
# | Type  : tristate
# | Prompt: IP set support
# |   Location:
# |     -> Networking support (NET [=y])
# |       -> Networking options
# | (1)     -> Network packet filtering framework (Netfilter) (NETFILTER [=y])
# |   Defined at net/netfilter/ipset/Kconfig:1
# |   Depends on: NET [=y] && INET [=y] && NETFILTER [=y]
# |   Selects: NETFILTER_NETLINK [=m]

# NETFILTER_XT_SET
# | Symbol: NETFILTER_XT_SET [=n]
# | Type  : tristate
# | Prompt: set target and match support
# |   Location:
# |     -> Networking support (NET [=y])
# |       -> Networking options
# |         -> Network packet filtering framework (Netfilter) (NETFILTER [=y])
# |           -> Core Netfilter Configuration
# | (1)         -> Netfilter Xtables support (required for ip_tables) (NETFILTER_XTABLES [=m])
# |   Defined at net/netfilter/Kconfig:644
# |   Depends on: NET [=y] && INET [=y] && NETFILTER [=y] && NETFILTER_XTABLES [=m] && IP_SET [=n] && NETFILTER_ADVANCED [=y]

# BASEDIR_CORE
# | Symbol: BASEDIR_CORE [=n]
# | Type  : tristate
# | Prompt: Generic Target Core Mod (TCM) and ConfigFS Infrastructure
# |   Location:
# | (1) -> Device Drivers
# |   Defined at drivers/target/Kconfig:2
# |   Depends on: SCSI [=y] && BLOCK [=y]
# |   Selects: CONFIGFS_FS [=y] && CRC_T10DIF [=y]

# ISCSI_BASEDIR
# | Symbol: ISCSI_BASEDIR [=n]
# | Type  : tristate
# | Prompt: Linux-iSCSI.org iSCSI Target Mode Stack
# |   Location:
# |     -> Device Drivers
# | (1)   -> Generic Target Core Mod (TCM) and ConfigFS Infrastructure (BASEDIR_CORE [=n])
# |   Defined at drivers/target/iscsi/Kconfig:1
# |   Depends on: BASEDIR_CORE [=n] && NET [=y]
# |   Selects: CRYPTO [=y] && CRYPTO_CRC32C [=y] && CRYPTO_CRC32C_INTEL [=n]

# SCSI_ISCSI_ATTRS
# | Symbol: SCSI_ISCSI_ATTRS [=n]
# | Type  : tristate
# | Prompt: iSCSI Transport Attributes
# |   Location:
# |     -> Device Drivers
# |       -> SCSI device support
# | (1)     -> SCSI Transports
# |   Defined at drivers/scsi/Kconfig:272
# |   Depends on: SCSI [=y] && NET [=y]
# |   Selects: BLK_DEV_BSGLIB [=n]
# |   Selected by: ISCSI_TCP [=n] && SCSI_LOWLEVEL [=y] && SCSI [=y] && INET [=y] || SCSI_CXGB3_ISCSI [=n] && SCSI_LOWLEVEL [=y] && SCSI [=y] && PCI [=y] && INET [=y] && (IPV6 [=y] || IPV6 [=y]=n)

# ISCSI_TCP
# | Symbol: ISCSI_TCP [=n]
# | Type  : tristate
# | Prompt: iSCSI Initiator over TCP/IP
# |   Location:
# |     -> Device Drivers
# |       -> SCSI device support
# | (1)     -> SCSI low-level drivers (SCSI_LOWLEVEL [=y])
# |   Defined at drivers/scsi/Kconfig:307
# |   Depends on: SCSI_LOWLEVEL [=y] && SCSI [=y] && INET [=y]
# |   Selects: CRYPTO [=y] && CRYPTO_MD5 [=m] && CRYPTO_CRC32C [=y] && SCSI_ISCSI_ATTRS [=n]

# ISCSI_BOOT_SYSFS
# | Symbol: ISCSI_BOOT_SYSFS [=n]
# | Type  : tristate
# | Prompt: iSCSI Boot Sysfs Interface
# |   Location:
# |     -> Device Drivers
# |       -> SCSI device support
# | (1)     -> SCSI low-level drivers (SCSI_LOWLEVEL [=y])
# |   Defined at drivers/scsi/Kconfig:331
# |   Depends on: SCSI_LOWLEVEL [=y] && SCSI [=y]
# |   Selected by: BE2ISCSI [=n] && SCSI_LOWLEVEL [=y] && PCI [=y] && SCSI [=y] && NET [=y] || SCSI_QLA_ISCSI [=n] && SCSI_LOWLEVEL [=y] && PCI [=y] && SCSI [=y] && NET [=y] || ISCSI_IBFT [=n] && AC

# KVM
# | Symbol: KVM [=n]
# | Type  : boolean
# | Prompt: Kernel-based Virtual Machine (KVM) support
# |   Location:
# | (1) -> Virtualization (VIRTUALIZATION [=y])
# |   Defined at arch/arm64/kvm/Kconfig:22
# |   Depends on: VIRTUALIZATION [=y] && OF [=y]
# |   Selects: MMU_NOTIFIER [=y] && PREEMPT_NOTIFIERS [=n] && HAVE_KVM_CPU_RELAX_INTERCEPT [=n] && HAVE_KVM_ARCH_TLB_FLUSH_ALL [=n] && KVM_MMIO [=n] && KVM_ARM_HOST [=n] && KVM_GENERIC_DIRTYLOG_READ

# VHOST_NET
# | Symbol: VHOST_NET [=n]
# | Type  : tristate
# | Prompt: Host kernel accelerator for virtio net
# |   Location:
# | (1) -> Virtualization (VIRTUALIZATION [=y])
# |   Defined at drivers/vhost/Kconfig:1
# |   Depends on: VIRTUALIZATION [=y] && NET [=y] && EVENTFD [=y] && (TUN [=y] || !TUN [=y]) && (MACVTAP [=m] || !MACVTAP [=m])
# |   Selects: VHOST [=n]

- `CONFIG_NAMESPACES=y`: 名前空間の基本機能を有効化。各種分離に必須
- `CONFIG_NET_NS=y`: ネットワーク名前空間を有効化。Pod毎にネットワーク空間を隔離するため
- `CONFIG_PID_NS=y`: プロセスID名前空間を有効化。コンテナ内でPIDを再利用し隔離するため
- `CONFIG_IPC_NS=y`: IPC名前空間を有効化。コンテナ間でSystem V IPC等を隔離するため
- `CONFIG_UTS_NS=y`: UTS名前空間を有効化。ホスト名等の隔離に必要
- `CONFIG_CGROUPS=y`: コントロールグループ(cgroups)機能を有効化。リソース制御の基本
- `CONFIG_CGROUP_PIDS=y`: プロセス数のcgroup制限を有効化。Pod毎のプロセス数制限に使用
- `CONFIG_CGROUP_CPUACCT=y`: CPU使用量の会計機能を有効化。K8sがCPU使用統計を取得するのに必要
- `CONFIG_CGROUP_SCHED=y`: CPUスケジューラのグループ拡張を有効化。CPUシェアやクォータ制御（CFS帯域制限）に必須
- `CONFIG_CPUSETS=y`: CPUセット機能を有効化。特定CPUコアへの割り当て（CPU Pinning）に利用
- `CONFIG_CFS_BANDWIDTH=y`: CPUのCFS帯域幅制限（CPUクォータ）を有効化。PodのCPU制限実装に必要
- `CONFIG_FAIR_GROUP_SCHED=y`: CFSのグループスケジューリングを有効化。cgroup単位の公平なCPU割当てに必要
- `CONFIG_RT_GROUP_SCHED=y`: RTスケジューリングのグループ制御を有効化。リアルタイム優先度のグループ管理（必要なら）
- `CONFIG_CGROUP_DEVICE=y`: デバイスアクセス制御を有効化。コンテナ毎のデバイス許可/禁止に必要
- `CONFIG_CGROUP_FREEZER=y`: プロセス停止のcgroup制御を有効化。Podの一時停止等に利用可能
- `CONFIG_MEMCG=y`: メモリコントローラを有効化。コンテナ毎のメモリ制限に必須
- `CONFIG_CGROUP_HUGETLB=y`: hugepagesのcgroup制御を有効化。巨大ページ使用量制限（必要なら）に対応
- `CONFIG_BLK_CGROUP=y`: ブロックIOのcgroup制御を有効化。ストレージIO帯域制限に必要
- `CONFIG_BLK_DEV_THROTTLING=y`: ブロックデバイスIOスロットルを有効化。IO制限（必要なら）に対応
- `CONFIG_CGROUP_PERF=y`: パフォーマンス計測のcgroup拡張を有効化。cgroup単位のperf統計取得に必要
- `CONFIG_NET_CLS_CGROUP=y`: ネットワーククラス分類のcgroup統合を有効化（オプション、必要に応じて）
- `CONFIG_CGROUP_NET_PRIO=y`: ネットワーク優先度制御のcgroup統合を有効化（オプション）
- `CONFIG_POSIX_MQUEUE=y`: POSIXメッセージキューを有効化。コンテナランタイムやアプリが使用する可能性があるため
- `CONFIG_KEYS=y`: キー管理機能を有効化。認証や暗号化（例: overlayFSの暗号化やCephなど使用時）に必要
- `CONFIG_USER_NS=y`: (推奨オプション) ユーザー名前空間を有効化。rootlessコンテナ等に必要（任意だが推奨）
- `CONFIG_SECCOMP=y`: (推奨オプション) Seccompを有効化。KubernetesのデフォルトSeccompプロファイルによるSyscall制限に必要
- `CONFIG_OVERLAY_FS=m`: (コンテナストレージ) OverlayFSを有効化。コンテナのレイヤードFS実現に必要（Docker/Containerdのoverlayストレージ)（モジュールとしてビルド可）
- 補足: 上記の多くはK3s内のcheck-configでも「Generally Necessary」または「Optional Features」として挙げられている項目です ￼特に`CONFIG_MEMCG（メモリcgroup）など一部はJetsonの標準カーネルでは無効の場合があり、Kubernetes動作のため有効化が必要です
またCONFIG_OVERLAY_FSはコンテナイメージのレイヤー実装に必須です。Jetson NanoのデフォルトカーネルでもDocker対応のため有効化（モジュール）されていますが、未設定の場合は有効にしてください`
- `CONFIG_VETH=y`: 仮想Ethernetデバイスを有効化。Pod用のvethペア作成に必須
- `CONFIG_BRIDGE=y`: ブリッジ機能を有効化。コンテナネットワークでブリッジを利用する際に必要
- `CONFIG_BRIDGE_NETFILTER=m`: ブリッジ上のトラフィックに対するiptablesフィルタを有効化。flannel等でブリッジ経由通信をフィルタ/NATする場合に必要（モジュール可）
- `CONFIG_NETFILTER=y`: Netfilterフレームワークを有効化。iptablesによるパケットフィルタ/NAT処理の基盤（モジュール可）
- `CONFIG_IP_NF_FILTER=m`: IPv4パケットフィルタ(iptables filterテーブル)を有効化。Kubernetesのサービスやネットワークポリシーで必要（モジュール可）
- `CONFIG_IP_NF_BASEDIR_MASQUERADE=m`: MASQUERADE（アドレス偽装）ターゲットを有効化。Podから外部へのSNATやflannelの外向きトラフィック処理に必須（モジュール可）
- `CONFIG_IP_NF_BASEDIR_REJECT=m`: REJECTターゲットを有効化。不要トラフィックの拒否に使用（例：不許可ポートのICMP拒否応答)（モジュール可）
- `CONFIG_IP_NF_NAT=m`: IPv4 NAT機能を有効化。KubernetesサービスのClusterIP⇔Pod IP変換やPod外部通信のSNATに必要（モジュール可）
- `CONFIG_NF_NAT=m`: 汎用NAT機能を有効化。上記とセットで必要（モジュール可）
- `CONFIG_NF_CONNTRACK=y`: コネクショントラッキングを有効化。iptablesの状態管理やNATに必須（自動選択される場合あり）。
- `CONFIG_NETFILTER_XT_MATCH_CONNTRACK=m`: iptables拡張マッチ: Conntrack状態をマッチするモジュールを有効化。kube-proxyのルールで使用（モジュール可）
- `CONFIG_NETFILTER_XT_MATCH_ADDRTYPE=m`: iptables拡張マッチ: アドレスタイプをマッチするモジュールを有効化。Serviceの外部/内部判定等で使用（モジュール可）
- `CONFIG_NETFILTER_XT_MATCH_COMMENT=m`: iptables拡張マッチ: コメント付与用（ルール識別用）。K3sデフォルトルールで使用（モジュール可）
- `CONFIG_NETFILTER_XT_MATCH_MULTIPORT=m`: iptables拡張マッチ: 複数ポート指定用。複数サービスポートのマッチに使用（モジュール可）
- `CONFIG_IP_SET=m`: ipset機能を有効化。大量IPアドレスの集合を効率的に扱うため、kube-proxyがエンドポイントやサービスCIDRをセット管理する際に必要（モジュール可）
- `CONFIG_NETFILTER_XT_SET=m`: iptablesでipsetを利用するためのモジュールを有効化。上記 `CONFIG_IP_SET` とセットで必要。（モジュール可）
- `CONFIG_IP_VS=m`: (IPVSモード用) IPVS (IP Virtual Server)を有効化。kube-proxyをIPVSモードで利用する場合に必要（モジュール可）
- (IPVSモード用) `CONFIG_IP_VS_PROTO_TCP/UDP`=y: IPVSでTCP/UDPを扱うプロトコルサポートを有効化
- (IPVSモード用) `CONFIG_IP_VS_RR=m`: IPVSのラウンドロビン・スケジューラを有効化（モジュール可）
- (IPVSモード用) `CONFIG_IP_VS_NFCT=y`: IPVSとNetfilterのConntrack連携を有効化
- `CONFIG_VXLAN=m`: VXLANオーバーレイネットワーク機能を有効化。flannelのVXLANバックエンドに必須（モジュール可）
- `CONFIG_INET_UDP_TUNNEL=y`: VXLANで使用するUDPトンネルサポートを有効化（`CONFIG_VXLAN選択時に自動有効化）。
- `CONFIG_XFRM=y`: (任意：暗号化ネットワーク用) IPsec等のトンネリング/暗号化に必要。flannelをIPsecモードで使用する場合に要求される ￼
- `CONFIG_INET_XFRM_MODE_TRANSPORT=y`
- `CONFIG_INET_ESP=m`: (任意：暗号化ネットワーク用) IPsec ESPプロトコルサポートを有効化。暗号化パケットの処理に必要（モジュール可）
- `CONFIG_CRYPTO_AEAD=m`: (任意：暗号化ネットワーク用) 暗号アルゴリズム。ネットワーク暗号化（IPsec/WireGuard）に必要（暗号化を使う場合有効化）
- `CONFIG_CRYPTO_GCM=m`
- `CONFIG_CRYPTO_GHASH=m`
- 補足: 上記のiptables関連オプションは、K3sのkube-proxyがiptablesモードでサービスルーティングする際に必要になります。 ￼に示されるように、Jetsonカーネルでは一部がモジュールとして提供されています。また`CONFIG_IP_SETはJetson系ではデフォルト無効の場合があり ￼、Longhorn等他機能にも影響するため必ず有効化してください。`
- VXLAN機能（`CONFIG_VXLAN）もJetson向けカーネルで有効化されていることを確認してくださいVXLANにより各ノード間でOverlayネットワークを構築し、Pod間通信が実現されます。
- `CONFIG_SCSI_ISCSI_ATTRS=y`: iSCSIイニシエータ属性を有効化。iSCSIセッション管理に必要（open`-iscsiが要求）
- `CONFIG_ISCSI_TCP=m`: iSCSI over TCPクライアント機能を有効化。LonghornノードがiSCSIターゲットに接続するため（open-iscsiで使用） ￼（モジュール可）
- `CONFIG_ISCSI_BASEDIR=m`: iSCSIターゲット(LIO)機能を有効化。LonghornでボリュームをエクスポートするノードがiSCSIターゲットとして振る舞うため ￼（モジュール可）
- LIOターゲットコアを有効化。上記`CONFIG_ISCSI_BASEDIRを利用するための共通ターゲット機能 ￼※CONFIG_ISCSI_BASEDIR有効化時に自動選択。
CONFIG_BASEDIR_CORE=y`
- `CONFIG_ISCSI_BOOT_SYSFS=y`: (任意) iSCSIブート用設定(sysfs)を有効化。通常のLonghorn動作には不要（iSCSI経由でのネットブート用途）。
- 補足: Longhornの動作には各ノードでiSCSIイニシエータが必要であり、Ubuntu環境ではopen-iscsiサービスによって管理されます。Jetson Nanoの標準カーネルではこれらiSCSI関連オプションがすべて無効になっているため ￼、Longhorn導入前にカーネル再構築で有効化が必要です。特にiscsi_tcpモジュール（`CONFIG_ISCSI_TCP）が存在しないとLonghornボリュームをアタッチできませんこれらはモジュールとしてビルドし、modprobe iscsi_tcp等で読み込む形でも動作可能です。`
- また、`CONFIG_ISCSI_BASEDIRとそれに依存するCONFIG_BASEDIR_COREを有効にすることで、Linux内蔵のiSCSIターゲット(LIO`)機能が利用可能になります。Longhornでは通常ユーザー空間でボリュームを管理しますが、将来的にカーネルのLIOターゲットを使う場合に備えて有効化しておくと良いでしょう有効化するとカーネルモジュールtarget_core_mod.koおよびiscsi_target_mod.koとして提供されます
- `CONFIG_KVM=y`: KVM (Kernel Virtual Machine)を有効化。ホスト側仮想化の基本機能（/dev/kvmデバイス） 
- `CONFIG_VHOST_NET=m`: Vhost-Netを有効化。仮想マシンのネットワークをカーネル空間で高速化するモジュール（virtio-net用） ￼（モジュール可）
- `CONFIG_TUN=m`: TUN/TAPデバイスを有効化。仮想マシンとホスト間を接続する仮想ネットワークインタフェース(tapデバイス)に必須。（モジュール可）
- `CONFIG_VIRTIO=y`: Virtio仮想デバイスフレームワークを有効化。以下の各種virtioデバイスサポートに必要。
- `CONFIG_VIRTIO_NET=m`: Virtioネットワークデバイスを有効化。仮想マシン用の高性能ネットワークインタフェース（Windows VMでも対応ドライバあり）。（モジュール可）
- `CONFIG_VIRTIO_BLK=m`: Virtioブロックデバイスを有効化。仮想マシンのディスクを効率よく扱うため（Windows VMでも対応ドライバあり）。（モジュール可）
- 補足: Jetson Nanoの標準カーネルではKVMサポートが無効化されており、そのままでは/dev/kvmが存在せず仮想マシンを起動できません ￼そのため上記の`CONFIG_KVMを有効にしてカーネルを再構築する必要があります。幸い、Jetson NanoのCPU（ARM` Cortex-A57）は仮想化拡張に対応しており、再構築によりKVMを利用可能ですKVM有効化時にはあわせて`CONFIG_VHOST_NETも有効化することが推奨されています。これは仮想NICのパケット処理をユーザ空間のQEMUではなくカーネルで行い、高速化するためのモジュールです ￼`
- また、Windows仮想マシンを運用する際にはvirtioデバイス（ネットワークやブロック）が使えると性能が向上します。上記のように`CONFIG_VIRTIO_NETやCONFIG_VIRTIO_BLKを有効にしておけば、Windows用のvirtioドライバを導入することで効率的なI/Oが可能になります。`
- なお、仮想マシンのネットワークブリッジには/dev/tunデバイス（Tun/Tapドライバ）が必要になるため、`CONFIG_TUNもモジュールで構いませんので有効化してください。

In [ ]:
diff -u "${TEGRA_KERNEL_OUT:?}"/.config{.old,.orig}
diff -u "${TEGRA_KERNEL_OUT:?}"/.config{.orig,}
diff -u "${TEGRA_KERNEL_OUT:?}"/.config{.old,}

In [ ]:
cd $JETSON_NANO_KERNEL_SOURCE/kernel/kernel-4.9
# https://qiita.com/ysakashita/items/566e082a5d060eef5046#%E3%82%AB%E3%83%BC%E3%83%8D%E3%83%AB%E3%81%AE%E3%82%B3%E3%83%B3%E3%83%91%E3%82%A4%E3%83%AB
#REGEX='CONFIG_(KVM|VHOST_NET|VHOST_VSOCK|VSOCKETS|MACVTAP|VIRTIO|CFQ_GROUP_IOSCHED|NETFILTER_XT_SET|IP_SET.*|NET_EMATCH_IPSET|SCSI_CONSTANTS|SCSI_LOGGING|SCSI_SCAN_ASYNC|SCSI_ISCSI_ATTRS|DM_BUFIO|DM_BIO_PRISON|DM_PERSISTENT_DATA|DM_THIN_PROVISIONING|IP_NF_BASEDIR_REDIRECT|CGROUP_HUGETLB|ISCSI)='
REGEX='CONFIG_(NAMESPACES|NET_NS|PID_NS|IPC_NS|UTS_NS|CGROUPS|CGROUP_PIDS|CGROUP_CPUACCT|CGROUP_SCHED|CPUSETS|CFS_BANDWIDTH|FAIR_GROUP_SCHED|RT_GROUP_SCHED|CGROUP_DEVICE|CGROUP_FREEZER|MEMCG|CGROUP_HUGETLB|BLK_CGROUP|BLK_DEV_THROTTLING|CGROUP_PERF|NET_CLS_CGROUP|CGROUP_NET_PRIO|POSIX_MQUEUE|KEYS|USER_NS|SECCOMP|OVERLAY_FS|VETH|BRIDGE|BRIDGE_NETFILTER|NETFILTER|IP_NF_FILTER|IP_NF_TARGET_MASQUERADE|IP_NF_TARGET_REJECT|IP_NF_NAT|NF_NAT|NF_CONNTRACK|NETFILTER_XT_MATCH_CONNTRACK|NETFILTER_XT_MATCH_ADDRTYPE|NETFILTER_XT_MATCH_COMMENT|NETFILTER_XT_MATCH_MULTIPORT|IP_SET|NETFILTER_XT_SET|IP_VS|IP_VS_PROTO_TCP|IP_VS_PROTO_UDP|IP_VS_RR|IP_VS_NFCT|VXLAN|INET_UDP_TUNNEL|XFRM|INET_XFRM_MODE_TRANSPORT|INET_ESP|CRYPTO_AEAD|CRYPTO_GCM|CRYPTO_GHASH|SCSI_ISCSI_ATTRS|ISCSI_TCP|ISCSI_TARGET|TARGET_CORE|ISCSI_BOOT_SYSFS|KVM|KVM_ARM_HOST|VHOST_NET|TUN|VIRTIO|VIRTIO_NET|VIRTIO_BLK)='
grep -E "$REGEX" $JETSON_NANO_KERNEL_SOURCE/kernel/kernel-4.9/arch/arm64/configs/tegra_defconfig
grep -E "$REGEX" $JETSON_NANO_KERNEL_SOURCE/kernel/kernel-4.9/arch/arm64/configs/tegra_defconfig | wc -l # => 43
zcat /proc/config.gz | grep -E "$REGEX"
zcat /proc/config.gz | grep -E "$REGEX" | wc -l # => 56
grep -E "$REGEX" "${TEGRA_KERNEL_OUT:?}"/.config
grep -E "$REGEX" "${TEGRA_KERNEL_OUT:?}"/.config | wc -l # => 67
# [ -f arch/arm64/configs/tegra_defconfig ] && ! grep 'CONFIG_KVM=y' $JETSON_NANO_KERNEL_SOURCE/kernel/kernel-4.9/arch/arm64/configs/tegra_defconfig &&
#   echo -e "CONFIG_KVM=y\nCONFIG_VHOST_NET=m\nCONFIG_VHOST_VSOCK=m" >>$JETSON_NANO_KERNEL_SOURCE/kernel/kernel-4.9/arch/arm64/configs/tegra_defconfig
cd $JETSON_NANO_KERNEL_SOURCE

In [ ]:
# Remove most generated files but keep the config and enough build support to build external modules
#make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" clean
# Remove all generated files + config + various backup files + remove editor backup and patch files
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" distclean

In [ ]:
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" olddefconfig  # 依存関係を自動修正
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" oldconfig     # 依存関係を修正 (修正値は対話的に聞かれる)

In [2]:
N=$(( $(nproc) + 1 ))
echo $N

bash: command not found: nproc
1


In [ ]:
# make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N prepare
# make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N modules_prepare

In [ ]:
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" WERROR=0 -j$N

In [ ]:
# Generates the Image that we're gonna place on /boot/Image
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N --output-sync=target zImage

In [ ]:
# Generates the drivers. This is needed because the old driver will not work with our new Image
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N --output-sync=target modules

In [ ]:
# Generates our modified device file trees
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N --output-sync=target dtbs

In [ ]:
# Installs the modules on the build folder ~/Linux_for_Tegra/source/public/build
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" INSTALL_MOD_PATH="${KERNEL_MODULES_OUT:?}" modules_install

In [ ]:
make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N --output-sync=target zImage && \
  make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N --output-sync=target modules && \
  make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N --output-sync=target dtbs && \
  make -C kernel/kernel-4.9/ ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" INSTALL_MOD_PATH="${KERNEL_MODULES_OUT:?}" modules_install

Now that we have our Image, the drivers and the file trees, we should override them, but before, make a manual backup of folders we're gonna change so you can rollback if something goes wrong.

In [ ]:
[ ! -d /boot.orig ] && sudo rsync -navh --delete /boot/ /boot.orig
[ ! -d /lib.orig ] && sudo rsync -navh --delete /lib/ /lib.orig

In [ ]:
# Reset
sudo rsync -avh --delete /lib.orig/ /lib
sudo rsync -avh --delete /boot.orig/ /boot

In [ ]:
cd "${KERNEL_MODULES_OUT:?}"/lib/
ls -lAF /lib/firmware
ls -lAF "${KERNEL_MODULES_OUT:?}"/lib/firmware
#sudo cp -r firmware /lib/firmware
rsync -n -rltDv "${KERNEL_MODULES_OUT:?}"/lib/firmware/ /lib/firmware
sudo rsync -rltDv "${KERNEL_MODULES_OUT:?}"/lib/firmware/ /lib/firmware

In [ ]:
ls -lAF /lib/modules/4.9.299-tegra
ls -lAF "${KERNEL_MODULES_OUT:?}"/lib/modules/4.9.299-tegra
#sudo cp -r modules /lib/modules
rsync -n -rltDv "${KERNEL_MODULES_OUT:?}"/lib/modules/ /lib/modules
sudo rsync -rltDv "${KERNEL_MODULES_OUT:?}"/lib/modules/ /lib/modules

Now we can `rsync` the files with the system ones (warning, untested, I used `sudo nautilus` and moved by hand on mine).

In [ ]:
# rsync -avh firmware /lib/firmware
# rsync -avh modules /lib/modules

Now we must also update the boot folder:

In [ ]:
cd "${TEGRA_KERNEL_OUT:?}"/arch/arm64/
ls -lAF /boot
ls -lAF "${TEGRA_KERNEL_OUT:?}"/arch/arm64/boot
rsync -nrltDv "${TEGRA_KERNEL_OUT:?}"/arch/arm64/boot/ /boot
sudo rsync -rltDv "${TEGRA_KERNEL_OUT:?}"/arch/arm64/boot/ /boot

In [ ]:
# rm -rf "${BASEDIR:?}"/kernel-*.tar.gz
rm -v "${BASEDIR:?}"/kernel*.tar.gz

# -a, --auto-compress
# -z, --gzip
# -h, --dereference
# -f, --file=ARCHIVE
# tar -acf "${BASEDIR:?}"/kernel-4.9.299-tegra-boot-lib.tar.gz -C "${TEGRA_KERNEL_OUT:?}"/arch/arm64 boot  -C "${KERNEL_MODULES_OUT:?}" lib

cd "${BASEDIR:?}"
tar -acf "${BASEDIR:?}"/kernel-4.9.299-tegra.tar.gz "${TEGRA_KERNEL_OUT:?}" ${JETSON_NANO_KERNEL_SOURCE:?}/kernel/kernel-4.9 "${KERNEL_MODULES_OUT:?}"/lib

find /lib/firmware /lib/modules /boot ! -user root

Notice that we copied all of the dtb files, there are many for different models, but just one that we should use. Run

In [ ]:
sudo dmesg | grep -i kernel

In [ ]:
sudo dmesg | grep -i kernel | grep DTS

to discover yours. Example of mine:

```log
[    0.236710] DTS File Name: /home/lz/Linux_for_Tegra/source/public/kernel/kernel-4.9/arch/arm64/boot/dts/../../../../../../hardware/nvidia/platform/t210/porg/kernel-dts/tegra210-p3448-0000-p3449-0000-a00.dts
```

Example of mine:

```log
[    0.207623] DTS File Name: /dvs/git/dirty/git-master_linux/kernel/kernel-4.9/arch/arm64/boot/dts/../../../../../../hardware/nvidia/platform/t210/porg/kernel-dts/tegra210-p3448-0000-p3449-0000-a02.dts
[    0.412171] DTS File Name: /dvs/git/dirty/git-master_linux/kernel/kernel-4.9/arch/arm64/boot/dts/../../../../../../hardware/nvidia/platform/t210/porg/kernel-dts/tegra210-p3448-0000-p3449-0000-a02.dts
```
```log
[    0.232381] DTS File Name: /dvs/git/dirty/git-master_linux/kernel/kernel-4.9/arch/arm64/boot/dts/../../../../../../hardware/nvidia/platform/t210/porg/kernel-dts/tegra210-p3448-0000-p3449-0000-a02.dts
[    0.440429] DTS File Name: /dvs/git/dirty/git-master_linux/kernel/kernel-4.9/arch/arm64/boot/dts/../../../../../../hardware/nvidia/platform/t210/porg/kernel-dts/tegra210-p3448-0000-p3449-0000-a02.dts
```
```log
[    0.231954] DTS File Name: /dvs/git/dirty/git-master_linux/kernel/kernel-4.9/arch/arm64/boot/dts/../../../../../../hardware/nvidia/platform/t210/porg/kernel-dts/tegra210-p3448-0000-p3449-0000-a02.dts
[    0.444484] DTS File Name: /dvs/git/dirty/git-master_linux/kernel/kernel-4.9/arch/arm64/boot/dts/../../../../../../hardware/nvidia/platform/t210/porg/kernel-dts/tegra210-p3448-0000-p3449-0000-a02.dts
```

Wait, wtf? Why this is a local file? I don't know what's happening, but this should show you which one is being used. You're gonna need its name. The file is already at `/boot`.

You might wonder that since we replaced all the device tree files on `/boot`, then it should load the modified one already. Somehow, in my case, it didn't. I think it has to do with the fact that it's loading a local one like shown above. If you know how to change this, open an issue please. Anyways, to bypass this, we have to inform the `/boot/extlinux/extlinux.conf` where to locate our file. Change from

```conf
TIMEOUT 30
DEFAULT primary

MENU TITLE L4T boot options

LABEL primary
      MENU LABEL primary kernel
      LINUX /boot/Image
      INITRD /boot/initrd
      APPEND ${cbootargs} quiet root=/dev/mmcblk0p1 rw rootwait rootfstype=ext4 loglevel=7 console=ttyS0,115200n8 console=tty0 fbcon=map:0 net.ifnames=0
```

to

```conf
TIMEOUT 30
DEFAULT primary

MENU TITLE L4T boot options

LABEL primary
      MENU LABEL primary kernel
      LINUX /boot/Image
      INITRD /boot/initrd
      FDT /boot/tegra210-p3448-0000-p3449-0000-a00.dtb
      APPEND ${cbootargs} quiet root=/dev/mmcblk0p1 rw rootwait rootfstype=ext4 loglevel=7 console=ttyS0,115200n8 console=tty0 fbcon=map:0 net.ifnames=0
```

that is, add the path to your dtb file. In my case, `FDT /boot/tegra210-p3448-0000-p3449-0000-a00.dtb`.

In [ ]:
ls -laF /boot/tegra210-p3448-0000-p3449-0000-a02.dt*
grep tegra210-p3448-0000-p3449-0000-a02 /boot/extlinux/extlinux.conf
# LABEL backup
#       MENU LABEL backup kernel
#       LINUX /boot.orig/Image
#       INITRD /boot.orig/initrd
#       APPEND ${cbootargs}


Note that you can add a second testing profile, which can be selected at boot time if you have a serial device to plug into the jetson nano like in this video https://www.youtube.com/watch?v=Kwpxhw41W50. When you boot you can select your second `LABEL` by typing its number. This is useful if you want to test different `Image`s without substituting the original one like we did.

Now reboot, and then run

In [ ]:
ls /dev | grep -E '^vhost|kvm|vsock'
# =>
# kvm
# vhost-net
# vhost-vsock
# vsock

 to confirm if the `kvm` file exists. This means it's working. You should also run

In [ ]:
ls  /proc/device-tree/interrupt-controller
# Doc:
# => compatible '#interrupt-cells' interrupt-controller interrupt-parent interrupts linux,phandle name phandle reg status
# Before:
# => compatible  '#interrupt-cells'   interrupt-controller   interrupt-parent   linux,phandle   name   phandle   reg   status
# After:
# => compatible  '#interrupt-cells'   interrupt-controller   interrupt-parent   linux,phandle   name   phandle   reg   status
# => compatible  '#interrupt-cells'   interrupt-controller   interrupt-parent   linux,phandle   name   phandle   reg   status
# => compatible  '#interrupt-cells'   interrupt-controller   interrupt-parent   linux,phandle   name   phandle   reg   status

and see that the node `interrupts`, which didn't exist before, was added. This means the irc interrupt activation worked.

In [ ]:
sudo dmesg | grep -i interrupt
# After:
# => [    0.000000] /interrupt-controller@60004000: 192 interrupts forwarded to /interrupt-controller
# => [    0.000000] /interrupt-controller@60004000: 192 interrupts forwarded to /interrupt-controller
# => [    0.000000] /interrupt-controller@60004000: 192 interrupts forwarded to /interrupt-controller

In [ ]:
sudo dmesg | grep -iE 'vhost|vsock|kvm|virt'

In [ ]:
lsmod | grep -E 'vhost|vsock|kvm|virt'
# =>
# vhost_net              15023  0
# vhost                  52361  1 vhost_net
# macvtap                21473  1 vhost_net
# =>
# vhost_net              15023  0
# vhost                  52361  1 vhost_net
# macvtap                21473  1 vhost_net
# =>
# vhost_vsock            13434  0
# vmw_vsock_virtio_transport_common    30778  1 vhost_vsock
# vsock                  36419  2 vhost_vsock,vmw_vsock_virtio_transport_common
# vhost_net              15023  0
# vhost                  52489  2 vhost_vsock,vhost_net
# macvtap                21537  1 vhost_net

In [ ]:
cat /etc/modules
# =>
# vhost_net

You can run qemu/firecracker now. I only tested with firecracker though.